In [2]:

#General dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as scistat
from datetime import datetime

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Install and import wrds
!pip install wrds
import wrds

#ScipyStats
from scipy.stats import mstats
from scipy.stats.mstats import winsorize


from sklearn.model_selection import TimeSeriesSplit
from scipy.stats.mstats import winsorize

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Upload the Xu/Kelly Data

The below workflow uploads the original Kelly study's data in chunks for further processing.  

In [3]:
filename = "C:/Users/andrew_lazzeri/Desktop/Project_Files/Data Upload/datashare.csv"
df_test = pd.read_csv(filename, nrows = 1)
col_conv = {c: np.float32 for c in df_test.columns}
col_conv['permno'] = np.int32
col_conv['DATE'] = np.int32 #May want to change to date time
del df_test 

mylist = []

for chunk in  pd.read_csv(filename, sep=',', chunksize=5000, error_bad_lines = False, engine='python', dtype=col_conv):
    mylist.append(chunk)

big_data = pd.concat(mylist, axis= 0)
del mylist

#### Upload Monthly Fama French Data to Obtain the Risk Free Rate for Excess Returns

In [4]:
from pandas.tseries.offsets import Day, MonthEnd

rf_df_path = "C:/Users/andrew_lazzeri/Desktop/Project_Files/Data Upload/F-F_Research_Data_5_Factors_2x3.csv"
risk_free_df = pd.read_csv(rf_df_path, header = 2, parse_dates = ['Unnamed: 0'])
risk_free_df = risk_free_df.iloc[0:714]
risk_free_df = risk_free_df[['Unnamed: 0', 'RF']]
risk_free_df.columns = ['Date', 'RiskFree']
risk_free_df['RiskFree'] = risk_free_df['RiskFree'].astype(float)/100
risk_free_df['Date'] = pd.to_datetime(risk_free_df['Date'], format='%Y%m', errors='coerce')
risk_free_df['Date'] = risk_free_df['Date'] + MonthEnd() 
risk_free_df = risk_free_df.set_index(['Date'])
risk_free_df

,RiskFree
Date,
1963-07-31,0.0027
1963-08-31,0.0025
1963-09-30,0.0027
1963-10-31,0.0029
1963-11-30,0.0027
...,...
2022-08-31,0.0019
2022-09-30,0.0019
2022-10-31,0.0023


#### Log on to WRDS

Here you have to establish a connection to WRDS and use your password.  I have a specific login and password setup I didn't include here so I didn't forget and send it out to the public.

In [5]:
#lazz0402
#Tignale04021986!

conn = wrds.Connection()

Enter your WRDS username [andrew_lazzeri]:lazz0402
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: n
You can create this file yourself at any time
with the create_pgpass_file() function.
Loading library list...
Done


#### The below functions append Compustat total returns to the original dataframe


In [6]:
#Extract the returns from Compustat
def where_string(permno_list):
    
    """
    Input: List of permnos (security specific identifiers) 
    -creates a list of permnos to search in the WRDS SQL query 
    -links them with an "OR permno =" which allows pulling multiple permnos 
    from a Compustat SQL search

    """
    
    ticker_str = "where permno = " + str(permno_list[0])
    for i in range(1, len(permno_list)):
        ticker_str += " OR permno = " + str(permno_list[i])
    return ticker_str

def return_pull(permno_list):
    
    """
    -Takes the permno_list and pulls the returns for each security in that list since 1957
    -Returns a pivoted dataframe with the permno as the columns, date rows, total return values
    """

    subtr = where_string(permno_list)
    SQL_str = "select permno, date, ret from crsp.msf " + subtr + " and date>='01/01/1957'"
    cstat_pull = conn.raw_sql(SQL_str, 
                     date_cols=['date'])
    return cstat_pull.pivot(values = 'ret', index = ['date'], columns = 'permno')

def fwd_return(df, m_fwd):
    """
    Takes a pivoted dataframe and calculates the forward returns m_fwd periods
    """
    
    iter_df = df.copy()
    iter_df = np.log(iter_df.shift(m_fwd)) - np.log(iter_df)
    return iter_df

def melt_df(df, m_string):
    """
    A simple helper function to melt the dataframe repeatedly
    """
    idx_list = list(df.index)
    melt_df = pd.melt(df)
    melt_df['date'] = (idx_list * df.shape[1])
    melt_df.columns = ['permno', str(m_string)+'m_fwd_ret', 'date']
    return melt_df

def return_merge(permno_list, risk_free_df):
    """
    -Pulls the return dataframe and then creates a cumulative indexed return for each security. 
    -calculates the forward returns over various horizons.  
    -Forward returns are then melted from a pivoted dataframes
    -Pivoted dataframes consolidated into a series of fwd return columns 
    """
    
    #Pull the returns information from Compustat and produce a cumulative total return index
    return_df =  return_pull(permno_list)
    return_df = return_df.reset_index()
    return_df['date'] = pd.to_datetime(return_df['date']) + MonthEnd() 
    return_df = pd.melt(return_df, id_vars = 'date')
    return_df = return_df.set_index(['date'])
    return_df = return_df.join(risk_free_df)
    return_df['ExcessRet'] = return_df['value'].astype(float) - return_df['RiskFree'].astype(float)
    return_df = return_df.reset_index()
    return_df.columns = ['date', 'permno', 'value', 'RiskFree', 'ExcessRet']
    return_df = return_df[['date', 'permno', 'ExcessRet']]
    return_df = return_df.reset_index().pivot_table(index = 'date', columns = 'permno', values = 'ExcessRet')
    return_df = return_df + 1
    c_return_df = return_df.cumprod()
    
    #Setup fwd returns
    fwd_1m_ret = fwd_return(c_return_df, -1)
    fwd_12m_ret = fwd_return(c_return_df, -12)
    
    #Melted dfs
    ret_m = melt_df(c_return_df, 0)
    fwd_1m_ret_m = melt_df(fwd_1m_ret, 1)
    fwd_12m_ret_m = melt_df(fwd_12m_ret, 12)
    
    #Consolidate dataframe
    final_df = ret_m
    df_list = [fwd_1m_ret_m, fwd_12m_ret_m]
    for ret_df in df_list: 
        final_df = final_df.merge(ret_df, left_on = ['permno', 'date'], right_on = ['permno', 'date'])

    return final_df.dropna(subset=['1m_fwd_ret'])

In [7]:
permno_list = list(set(big_data['permno'].to_list()))
len(permno_list)

32793

In [8]:
append_df = return_merge(permno_list, risk_free_df)
append_df.to_csv('C:/Users/andrew_lazzeri/Desktop/Project_Files/Data Upload/return dfs/append_df.csv')

In [9]:
"""
def small_pulls(permno_list, risk_free_df):
    return_df = pd.DataFrame()
    len_permno = len(permno_list)
    samples = list(range(0, len_permno, 10000))
    for i in range(1, len(samples)):
        iter_returns = return_merge(permno_list[samples[i-1]:samples[i]], risk_free_df)
        return_df = pd.concat([return_df, iter_returns])
         
    return return_df

append_df = small_pulls(permno_list, risk_free_df)

append_df.to_csv('C:/Users/andrew_lazzeri/Desktop/Project_Files/Data Upload/return dfs/append_df.csv')
"""

"\ndef small_pulls(permno_list, risk_free_df):\n    return_df = pd.DataFrame()\n    len_permno = len(permno_list)\n    samples = list(range(0, len_permno, 10000))\n    for i in range(1, len(samples)):\n        iter_returns = return_merge(permno_list[samples[i-1]:samples[i]], risk_free_df)\n        return_df = pd.concat([return_df, iter_returns])\n         \n    return return_df\n\nappend_df = small_pulls(permno_list, risk_free_df)\n\nappend_df.to_csv('C:/Users/andrew_lazzeri/Desktop/Project_Files/Data Upload/return dfs/append_df.csv')\n"

#### Once run, append the fwd total return data to the feature data

In [10]:
big_data['DATE'] = pd.to_datetime(big_data['DATE'])

#Join the two dataframes
big_data = big_data.merge(append_df, how = 'left', left_on = ['permno', 'DATE'], right_on = ['permno', 'date'])

In [11]:
big_data.to_csv('C:/Users/andrew_lazzeri/Desktop/Project_Files/Data Upload/final_cross_section_df.csv')